In [1]:
!pip install datasets peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 24.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "EMBEDDIA/sloberta"

In [4]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_RTE")

Generating train split:   0%|          | 0/232 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/29 [00:00<?, ? examples/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

In [6]:
id2label = {0: "not_entailment", 1: "entailment"}
label2id = {"not_entailment": 0, "entailment": 1}

In [7]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"
LABEL_COL = "label"

def preprocess_function(examples):
    context_inputs = [f"{CONTEXT_COL} : {x} " for x in examples[CONTEXT_COL]]
    hypothesis_inputs = [f"{HYPOTHESIS_COL} : {x} " for x in examples[HYPOTHESIS_COL]]
    inputs = [context_input + hypothesis_input for context_input, hypothesis_input in zip(context_inputs, hypothesis_inputs)]

    tokenized_examples = tokenizer(inputs, truncation=True)
    tokenized_examples[LABEL_COL] = [label2id[label] for label in examples[LABEL_COL]]

    return tokenized_examples

In [8]:
tokenized_dataset = dataset.map(preprocess_function, remove_columns=['idx', 'hypothesis', 'premise'], batched=True)

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [9]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
from transformers import set_seed

set_seed(42)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(id2label), id2label=id2label, label2id=label2id
)
model.config.use_cache = False

config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

new_model_name = "fine_tuned_rte_sloberta"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.005,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.703500,0.732206,0.413793,0.242220
100,0.703200,0.693545,0.413793,0.242220
150,0.699800,0.702104,0.413793,0.242220
200,0.697000,0.682826,0.586207,0.433283
250,0.696800,0.683704,0.586207,0.433283
300,0.696500,0.689685,0.586207,0.433283
350,0.693400,0.686912,0.586207,0.433283
400,0.687800,0.689777,0.689655,0.691160


TrainOutput(global_step=400, training_loss=0.6972385787963867, metrics={'train_runtime': 60.0734, 'train_samples_per_second': 53.268, 'train_steps_per_second': 6.659, 'total_flos': 268295165372640.0, 'train_loss': 0.6972385787963867, 'epoch': 13.793103448275861})

In [15]:
trainer.evaluate()

{'eval_loss': 0.689776599407196,
 'eval_accuracy': 0.6896551724137931,
 'eval_f1': 0.6911598746081503,
 'eval_runtime': 0.1325,
 'eval_samples_per_second': 218.848,
 'eval_steps_per_second': 30.186,
 'epoch': 13.793103448275861}

In [16]:
trainer.push_to_hub(new_model_name)

events.out.tfevents.1716495723.7bb630a2356a.939.0:   0%|          | 0.00/9.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

events.out.tfevents.1716495783.7bb630a2356a.939.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/fine_tuned_rte_sloberta/commit/62f849f7bed5cfb0902c736025f8d65a9bd245de', commit_message='fine_tuned_rte_sloberta', commit_description='', oid='62f849f7bed5cfb0902c736025f8d65a9bd245de', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
from transformers import pipeline

adapter_name = "lenatr99/" + new_model_name

# Example
text="premise : Potem ko je sklenila zavezništvo z glavno proti-sirijsko koalicijo, so njegovo ženo Strido na nedavnih volitvah izvolili v parlament. hypothesis : Strido so izvolili v parlament."

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/800k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

[{'label': 'not_entailment', 'score': 0.5063579082489014}]

In [17]:
# GPU RAM = 3.4 GB